In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
#ws = Workspace.get(name="quick-starts-ws-234286")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-235672
Azure region: westeurope
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-235672


In [60]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cluster_name = "project-cluster"

try:
    compute_target= ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

In [37]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "project_cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
from azure.ai.ml.entities import AmlCompute

cluster_basic = AmlCompute(
    name="basic-example",
    type="amlcompute",
    size="STANDARD_DS2_v2",
    location="westus",
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=120,
)
ml_client.begin_create_or_update(cluster_basic).result()
### YOUR CODE HERE ###

ModuleNotFoundError: No module named 'azure.ai'

In [64]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C':choice(0.01,0.1,1,10,20,50,100,200,500), '--max_iter':choice(50,100,200)
    }
    )

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15,delay_evaluation=5)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig("./", "train.py",  environment=sklearn_env, compute_target=compute_target)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    policy=policy,
                                    run_config=src,
                                    )

In [65]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

#hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cae69bc7-2201-4a02-94fd-8acf3f331cef
Web View: https://ml.azure.com/runs/HD_cae69bc7-2201-4a02-94fd-8acf3f331cef?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-235672/workspaces/quick-starts-ws-235672&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-13T15:04:23.436323][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-06-13T15:04:23.9007404Z][SCHEDULER][INFO]Scheduling job, id='HD_cae69bc7-2201-4a02-94fd-8acf3f331cef_0' 
[2023-06-13T15:04:24.0052204Z][SCHEDULER][INFO]Scheduling job, id='HD_cae69bc7-2201-4a02-94fd-8acf3f331cef_1' 
[2023-06-13T15:04:24.1522943Z][SCHEDULER][INFO]Scheduling job, id='HD_cae69bc7-2201-4a02-94fd-8acf3f331cef_2' 
[2023-06-13T15:04:24.291862][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-06-13T15:04:24.3760949Z][SCHEDULER][INFO]Scheduling job, id='HD_cae69bc7-2201-4a02-94fd-8acf3f331c

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

### YOUR CODE HERE ###


print("best run details :", best_run.get_details())
print('\n')
print("best run file names :", best_run.get_file_names())
print('\n')
print("best run metrics :", best_run.get_metrics())

model = best_run.register_model(model_name='hyperdrive_log_reg_best_run', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy" : best_run.get_metrics()["Accuracy"]})

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

tbds = TabularDatasetFactory.from_delimited_files(url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(tbds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task='classification', 
    primary_metric='accuracy',
    training_data=tbds,
    label_column_name='y'
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [2]:
# Submit your automl run
automl_run = exp.submit(config=automl_config, show_output=True)
automl_run.wait_for_completion()

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)
print(automl_best_run_onnx.get_metrics())
OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './/best_model.onnx' )
